In [1]:
import time
from collections import defaultdict

In [2]:
import pandas as pd
import numpy as np

In [3]:
from hybrid_search_engine.index import build_index_from_df
from hybrid_search_engine.search_engine import SearchEngine

d:\projects\hse\hybrid-search-engine\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
meta_df = pd.read_csv("data/metadata.csv")

In [5]:
meta_df.head(5)

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
2,le0ogx1s,NaN,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...


In [6]:
# sample_df = meta_df.sample(frac=0.01).copy(deep=True)
# sample_df.index = np.arange(len(sample_df))
# sample_df.set_index("cord_uid", inplace=True)

sample_df = meta_df.copy(deep=True)

In [7]:
sample_df

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
2,le0ogx1s,NaN,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51073,4xhcamks,e14e7f6ffed1a74954167e4fb52d1b2f156171cd,Elsevier,Prevalence of antibodies against transmissible...,10.1016/j.vas.2018.11.003,PMC7104163,NaN,els-covid,Abstract Transmissible gastroenteritis (TGE) i...,2019-06-30,"Valkó, Anna; Bálint, Ádám; Bozsa, Ágnes; Cságo...",Veterinary and Animal Science,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.vas.2018.11.003
51074,z2rqz3jx,9550137f5e105d2dbc14d91659a8c3cdd815c9b1,Elsevier,Can we increase public awareness without creat...,10.1016/j.pec.2013.10.023,PMC7119035,24284163.0,els-covid,NaN,2014-02-28,"Al Turki, Yousef Abdullah",Patient Education and Counseling,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.pec.2013.10.023
51075,wncuc903,2b3a3e3956afd65a8fa11d3867da3e58f8915514,Elsevier,Fast degrading polyesters as siRNA nano-carrie...,10.1016/j.jconrel.2008.06.010,PMC7125568,18619502.0,els-covid,Abstract A potential siRNA carrier for pulmona...,2008-12-18,"Nguyen, Juliane; Steele, Terry W.J.; Merkel, O...",Journal of Controlled Release,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.jconrel.2008.06.010
51076,4x1ckr2i,a67dae6e5f2b58a6f16aaed4011c694f9e64dd88,Elsevier,eIF3f: A central regulator of the antagonism a...,10.1016/j.biocel.2013.06.001,PMC7108353,23769948.0,els-covid,Abstract The eukaryotic initiation factor 3 su...,2013-10-31,"Sanchez, Anthony M.J.; Csibi, Alfredo; Raibon,...",The International Journal of Biochemistry & Ce...,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.biocel.2013.06.001


In [9]:
LOAD_FROM_FILE = False

In [10]:
text_cols = ["title", "abstract"]
id_column = "cord_uid"

if LOAD_FROM_FILE:
    index_df = pd.read_json("data/index_df_cs.json", orient="records")
    documents_df = pd.read_json("data/documents_df_cs.json", orient="records")
    
    index_df["token vector"] = index_df["token vector"].apply(np.array)
    
    for col in text_cols:
        index_df[col] = index_df[col].apply(np.array, dtype=np.int32)
        index_df[f"{col} TF"] = index_df[f"{col} TF"].apply(np.array, dtype=np.int32)
else:
    start_time = time.time()

    index_df, documents_df = build_index_from_df(sample_df, columns=text_cols, id_column=id_column, filtering_columns=[], lower=False)

    execution_time = time.time() - start_time

    print("--- %s seconds ---" % execution_time)

--- 1020.2104749679565 seconds ---


In [16]:
index_df.to_json("data/index_df_cs.json", orient="records")
documents_df.to_json("data/documents_df_cs.json", orient="records")

In [17]:
config = {}
config["lower"] = False
engine = SearchEngine(index=index_df, documents_df=documents_df, columns=text_cols, config=config)

In [22]:
query = "COVID symptoms"

In [23]:
result = engine.find(query=query)

D:\Projects\HSE\Hybrid-Search-Engine\hybrid_search_engine\search_engine.py:125: RuntimeWarning: invalid value encountered in true_divide
  v = np.array([c / np.linalg.norm(c) for c in v])


In [24]:
result.head(50)

,similarity
document_id,
wqutxi9c,3.867128
zmxmo1q8,3.359675
7t25qas0,3.189192
ba2amtdp,3.147092
6m5jlv0l,3.110481
28dv59bp,3.087108
bu2w2ox6,2.858329
m00gcci4,2.768398
r619rlhv,2.644661


In [25]:
meta_df.loc[meta_df["cord_uid"] == "wqutxi9c"]["abstract"].values

array(['Abstract Using the 2009 Swine Flu outbreak as a contemporary example of pandemic fears, this study examined the relationship between various symptoms related to anxiety sensitivity and Swine Flu fears. It was hypothesized that both obsessive–compulsive (OC) beliefs and OC symptoms would significantly predict Swine Flu fears. It was also hypothesized that symptoms of anxiety, including measures of anxiety sensitivity and disgust sensitivity would significantly mediate the relationship between both OC beliefs and OC symptoms and Swine Flu fears. A total of 393 undergraduate students completed measures of Swine Flu fears, anxiety sensitivity, OC beliefs and symptoms, and disgust sensitivity. It was found that both OC beliefs and OC symptoms significantly predicted Swine Flu fears. While disgust sensitivity significantly mediated the relationship between both OC beliefs and OC symptoms and Swine Flu fears using the Sobel test, anxiety sensitivity was a significant mediator only for

In [ ]:
sample_df.loc[result.index.values.tolist()]["abstract"][0]

In [ ]:
engine.nlp_engine("covid").vector

In [ ]:
engine.nlp_engine("sars").vector

In [ ]:
engine.nlp_engine("virus").similarity(engine.nlp_engine("virion"))

In [ ]:
from hybrid_search_engine.utils import text_processing as processing

In [ ]:
processing.split_camel_case("COVID")

In [ ]:
processing.process_string("COVID")

In [ ]:
processing.process_string("covid")

In [ ]:
index_df["token"]

In [ ]:
import spacy

from scispacy.abbreviation import AbbreviationDetector

nlp = spacy.load("en_core_sci_lg")

In [ ]:
abbradd_pipeiation_pipe = AbbreviationDetector(nlp)
nlp.add_pipe(abbreviation_pipe)

In [ ]:
doc = nlp("Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily.")

print("Abbreviation", "\t", "Definition")
for abrv in doc._.abbreviations:
    print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")

In [ ]:
s = "Dual Clutch Transimission (HJKLK) is a racing spec gearbox for faster gear shifting"

In [ ]:
s = "Coronavirus Desease (COVID) is very infectious desease"

In [ ]:
doc = nlp(s)

print("Abbreviation", "\t", "Definition")
for abrv in doc._.abbreviations:
    print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")

In [ ]:
doc

In [ ]:
doc.vector

In [ ]:
nlp("").vector.shape[0]

In [26]:
"ncov" in engine.nlp_engine.vocab

True

In [27]:
"sars" in engine.nlp_engine.vocab

True

In [29]:
engine.nlp_engine("sars").vector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [ ]:
if token not in nlp or np.isclose(nlp(token).vector, np.zeros(200)):
    pass

In [37]:
test_read_index_df = pd.read_csv("data/index_df.tsv", sep="\t")

In [ ]:
test_read_index_df["token vector"] =

In [39]:
test_read_index_df.head(5)

,token,token vector,title,title TF,abstract,abstract TF
0,airborne,[-2.63176206e-02 2.83739846e-02 -3.84504125e-...,[ 0 769 1850 1887 1904 2169 3153 43...,[1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,[ 0 769 863 1161 1390 1850 1904 21...,[ 3 1 2 1 1 5 1 6 2 1 2 4 5 1 1 ...
1,assay,[ 0.09331236 -0.06822105 -0.00424979 0.094302...,[ 0 46 48 76 148 215 294 5...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,[ 0 26 46 ... 51058 51062 51064],[1 2 2 ... 2 1 1]
2,detection,[ 0.00613616 -0.13328402 0.04559763 0.011014...,[ 0 17 43 ... 51015 51018 51040],[2 1 1 ... 1 1 1],[ 0 12 29 ... 51023 51035 51069],[4 1 1 ... 1 2 3]
3,effect,[ 0.02572836 -0.04476785 0.08602881 0.021475...,[ 0 66 112 ... 50955 50964 51020],[1 1 1 ... 1 1 1],[ 0 9 20 ... 51012 51020 51064],[1 3 1 ... 1 2 1]
4,irradiation,[-6.50346279e-02 -2.64813341e-02 -9.19022411e-...,[ 0 7634 9114 10385 15439 18131 23085 269...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1],[ 0 1010 1499 2931 3043 3209 3890 39...,[3 2 1 3 3 1 1 1 1 1 1 2 2 1 1 2 3 1 1 1 1 1 1...


In [40]:
index_df

,token,token vector,title,title TF,abstract,abstract TF
0,airborne,"[-0.02631762, 0.028373985, -0.038450412, -0.06...","[0, 769, 1850, 1887, 1904, 2169, 3153, 4398, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ...","[0, 769, 863, 1161, 1390, 1850, 1904, 2169, 23...","[3, 1, 2, 1, 1, 5, 1, 6, 2, 1, 2, 4, 5, 1, 1, ..."
1,assay,"[0.09331236, -0.068221055, -0.0042497874, 0.09...","[0, 46, 48, 76, 148, 215, 294, 523, 551, 583, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 26, 46, 48, 70, 76, 84, 93, 101, 119, 128,...","[1, 2, 2, 1, 2, 2, 1, 2, 1, 4, 1, 1, 1, 6, 9, ..."
2,detection,"[0.0061361627, -0.13328402, 0.045597628, 0.011...","[0, 17, 43, 46, 48, 64, 83, 139, 147, 148, 176...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 12, 29, 46, 58, 64, 71, 80, 103, 110, 113,...","[4, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 11,..."
3,effect,"[0.02572836, -0.04476785, 0.08602881, 0.021475...","[0, 66, 112, 179, 212, 214, 254, 272, 282, 306...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 9, 20, 36, 42, 45, 50, 56, 94, 95, 97, 98,...","[1, 3, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, ..."
4,irradiation,"[-0.06503463, -0.026481334, -0.09190224, 0.058...","[0, 7634, 9114, 10385, 15439, 18131, 23085, 26...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1010, 1499, 2931, 3043, 3209, 3890, 3991, ...","[3, 2, 1, 3, 3, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, ..."
...,...,...,...,...,...,...
107443,92log10,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],[],[51069],[1]
107444,9894,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],[],[51069],[1]
107445,9931,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],[],[51069],[1]
107446,atrogin,"[0.05800455, 0.09768766, 0.11903231, 0.0745232...",[],[],[51076],[3]


In [41]:
index_df.to_json("data/index_df.json", orient="records")

In [60]:
test_read_index_df_json = pd.read_json("data/index_df.json", orient="records")
test_read_documents_df_json = pd.read_json("data/documents_df.json", orient="records")

In [54]:
test_read_index_df_json["token vector"] = test_read_index_df_json["token vector"].apply(np.array)
test_read_index_df_json["title"] = test_read_index_df_json["title"].apply(np.array, dtype=np.int32)
test_read_index_df_json["abstract"] = test_read_index_df_json["abstract"].apply(np.array, dtype=np.int32)
test_read_index_df_json["title TF"] = test_read_index_df_json["title TF"].apply(np.array, dtype=np.int32)
test_read_index_df_json["abstract TF"] = test_read_index_df_json["abstract TF"].apply(np.array, dtype=np.int32)


In [61]:
test_read_documents_df_json.head(5)

,cord_uid,title Norm,abstract Norm
0,xqhn0vbp,0.288675,0.078087
1,gi6uaa83,0.447214,0.171499
2,le0ogx1s,0.447214,0.235702
3,fy4w7xz8,0.333333,0.061199
4,0qaoam29,0.447214,0.076472


In [62]:
documents_df

,cord_uid,title Norm,abstract Norm
0,xqhn0vbp,0.288675,0.078087
1,gi6uaa83,0.447214,0.171499
2,le0ogx1s,0.447214,0.235702
3,fy4w7xz8,0.333333,0.061199
4,0qaoam29,0.447214,0.076472
...,...,...,...
51073,4xhcamks,0.377964,0.111803
51074,z2rqz3jx,0.353553,0.000000
51075,wncuc903,0.333333,0.079809
51076,4x1ckr2i,0.353553,0.089803


In [63]:
!pip install torch===1.4.0 torchvision===0.5.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu101/torch-1.4.0-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/7d/3e/2b5ddf744226159dc90a52f0d044c0de7c5ca4f42d12a350a674ebb6fb2a/torchvision-0.5.0-cp37-cp37m-win_amd64.whl


You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [13]:
import spacy


nlp = spacy.load("en_core_web_lg")

OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [14]:
!python -m spacy download en_core_web_lg

^C


In [3]:
doc = "SARS is a dangerous disease"

In [19]:
w = "covid"
v = nlp(w).vector
print(v)
# print(doc.vector)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


Exception:
Traceback (most recent call last):
  File "D:\Projects\HSE\Hybrid-Search-Engine\lib\site-packages\pip\_vendor\urllib3\response.py", line 360, in _error_catcher
    yield
  File "D:\Projects\HSE\Hybrid-Search-Engine\lib\site-packages\pip\_vendor\urllib3\response.py", line 442, in read
    data = self._fp.read(amt)
  File "C:\Program Files\Python37\lib\http\client.py", line 457, in read
    n = self.readinto(b)
  File "C:\Program Files\Python37\lib\http\client.py", line 501, in readinto
    n = self.fp.readinto(b)
  File "C:\Program Files\Python37\lib\socket.py", line 589, in readinto
    return self._sock.recv_into(b)
  File "C:\Program Files\Python37\lib\ssl.py", line 1071, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Program Files\Python37\lib\ssl.py", line 929, in read
    return self._sslobj.read(len, buffer)
socket.timeout: The read operation timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):

In [6]:
t = doc[0]

In [8]:
ner = nlp.create_pipe("ner")

In [9]:
ner.labels

()

In [12]:
ner(nlp("COVID is a dangerous disease"))

ValueError: [E109] Model for component 'ner' not initialized. Did you forget to load a model, or forget to call begin_training()?

In [14]:
index_df[index_df.token.str.contains("HCo")]["token"].values

array(['HCoV', 'HCoVs', 'HCoVOC43', 'MethodsHCoV', 'ResultsHCoV',
       'rHCoVs', 'HCoVHKU1'], dtype=object)

In [36]:
from nltk.stem import WordNetLemmatizer

In [37]:
l = WordNetLemmatizer()

In [38]:
l.lemmatize("HCoV")

'HCoV'

In [26]:
import spacy

In [27]:
nlp = spacy.load("en_core_sci_lg")

In [35]:
COVID_TERMS = ['covid', 'sars-?n?cov-?2','2019-ncov', 'novel coronavirus', 'sars coronavirus 2', 'sars']
COVID_SEARCH = f".*({'|'.join(COVID_TERMS)})"

In [31]:
for term in COVID_TERMS:
    print(f"{term}\t{(nlp(term).vector != np.zeros(200)).all()}")

covid	False
sars-?n?cov-?2	False
2019-ncov	False
novel coronavirus	True
sars coronavirus 2	True


In [30]:
nlp("covid").vector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [39]:
index_df[index_df.token.str.match(COVID_SEARCH, case=False)].head(50)

,token,token vector,title,title TF,abstract,abstract TF
215,SARS,"[0.028603343, 0.12526345, 0.0014840632, 0.0094...","[4, 5, 6, 9, 13, 14, 18, 19, 20, 23, 25, 26, 3...","[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, ...","[3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 18, 19, 20,...","[13, 7, 5, 3, 2, 13, 1, 1, 3, 2, 4, 3, 11, 1, ..."
11714,SARSCoV,"[0.20671473, 0.018655937, -0.050014507, -0.079...","[33191, 40290]","[1, 1]","[593, 15346, 15483, 25150, 27425, 30944, 33170...","[4, 1, 1, 2, 1, 1, 1, 1, 1, 5, 2, 1]"
14599,hSARS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],[],[889],[1]
15134,SARSpp,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],[],"[947, 22211]","[2, 3]"
21760,rSARS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],[],"[1842, 17856, 18581, 20477, 39296, 48332, 4838...","[7, 6, 4, 3, 5, 4, 3, 1]"
23424,recSARS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],[],[2130],[4]
23425,vSARS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],[],[2130],[2]
46769,Boussarsar,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],[],[7858],[2]
48457,Sarshor,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],[],[7858],[1]
53232,SARSr,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[32810],[1],"[9206, 10945, 11260, 15649, 16372, 18211, 1884...","[1, 1, 2, 2, 1, 1, 27, 15, 3, 1, 2, 2, 2, 1, 1..."
